In [1]:
from urllib.request import urlopen
from urllib.parse import urlencode
import urllib3
import json
import pandas as pd
from time import sleep
from io import StringIO
import os
from enum import Enum
from concurrent.futures import ThreadPoolExecutor
import numpy as np

## Hydrology API code

In [32]:
class Measure(Enum):
    LEVEL = 'level'
    FLOW = 'flow'
    RAINFALL = 'rainfall'

class HydrologyApi:
    API_BASE_URL = "https://environment.data.gov.uk/hydrology/"
    DATA_DIR = "data"
    
    units = {
        Measure.LEVEL: 'i-900-m-qualified',
        Measure.FLOW: 'i-900-m3s-qualified',
        Measure.RAINFALL: 't-900-mm-qualified',
    }
    
    def __init__(self, max_threads):
        self.http = urllib3.PoolManager(maxsize=max_threads)
        self.thread_pool = ThreadPoolExecutor(max_workers=max_threads)
    
    def get_stations_on_river(self, river):
        api_url = self.API_BASE_URL + 'id/stations'
        result = urlopen(
            api_url + '?' + urlencode({'riverName': river})).read().decode('utf-8')
        data = json.loads(result)
        print(data)
        return pd.DataFrame(data['items'])
    
    def get_stations_close_to_with_measure(self, lat, lon, radius, measure: Measure, limit=100):
        api_url = self.API_BASE_URL + 'id/stations'
        
        result = self.http.request(
            'GET',
            api_url,
            fields={
                'observedProperty': measure.value,
                'lat': lat,
                'long': lon,
                'dist': radius,
                'status.label':'Active',
                '_limit': limit
            }
        ).data.decode('utf-8')
        data = json.loads(result)
        return pd.DataFrame(data['items'])
        
    
    def get_measure(self, measure: Measure, station_id: str, start=None):
        api_url = self.API_BASE_URL + f"id/measures/{station_id}-{measure.value}-{HydrologyApi.units[measure]}/readings"
        # result = urlopen(api_url).read().decode('utf-8')
        result = self.http.request(
            'GET',
            api_url,
            fields={}
                | ({
                    'mineq-date': start.strftime('%Y-%m-%d')
                } if start is not None else {}),
        ).data.decode('utf-8')
        data = json.loads(result)
        return pd.DataFrame(data['items'])
    
    def _batch_request(self, api_url):
        status = "Pending"

        while status in ("Pending", "InProgress"):
            print(f"Making request to: {api_url}")
            
            request = self.http.request(
                'GET', 
                api_url, 
                headers={
                    'Accept-Encoding': 'gzip'
                }
            )
            content_type = request.headers['Content-Type']

            if content_type == 'text/csv':
                if len(request.data) == 0:
                    print('Got empty CSV')
                    return None
                buffer = StringIO(request.data.decode('utf-8'))
                return pd.read_csv(buffer, low_memory=False)
            
            assert content_type in (
                'application/json',
                'application/json;charset=UTF-8'), f"Unexpected content type: {content_type}"

            data = json.loads(request.data.decode('utf-8'))
            status = data["status"]

            if status == "Pending":
                print(f"Query is pending")
                pos_in_queue = data["positionInQueue"]
                print(f"Position in queue: {pos_in_queue}")
                eta = data["eta"] / 1000
                print(f"Estimated completion: {eta}")
                sleep(eta * 1.1)

            elif status == "InProgress":
                print(f"Query in progress")
                eta = data["eta"] / 1000
                print(f"Estimated completion: {eta}")
                sleep(eta * 1.1)

            elif status in ("Complete", "Completed"):
                print(f"Query completed: {data}")
                csv_url = data["dataUrl"] if "dataUrl" in data else data["url"]
                return pd.read_csv(csv_url, low_memory=False)

            elif status == "Failed":
                raise Exception(f"Query failed, response: {data}")

            else:
                raise Exception(f"Unknown status: {data['status']}")
    
    def batch_get_measure(self, measure: Measure, station_id):
        try:
            api_url = self.API_BASE_URL + \
                f"data/batch-readings/batch/?measure={station_id}-{measure.value}-{HydrologyApi.units[measure]}"
                
            return self._batch_request(api_url)
        except Exception as e:
            print(f"Failed to get data for station: {station_id}, {e}")
            return None
        
    def batch_get_measure_on_river(self, measure: Measure, river):
        stations = self.get_stations_on_river(river)
        return self.batch_get_measure_from_stations(measure, stations)
        
    def batch_get_measure_from_stations(self, measure: Measure, stations):
        data = pd.DataFrame()
        threads = [
            self.thread_pool.submit(
                self.batch_get_measure, measure, station_id)
            for station_id in stations['notation'].values
        ]
        
        for thread, (station_id, station_name) in zip(threads, stations[['notation', 'label']].values):
            new_data = thread.result()
            if new_data is None:
                print(f"No new data for station: {station_name}")
                continue
            new_data = new_data.drop(columns=['measure', 'date', 'qcode', 'completeness'])
            new_data['station'] = station_name
            new_data['station'] = new_data['station'].astype('category')
            new_data['dateTime'] = pd.to_datetime(new_data['dateTime'])
            new_data['value'] = new_data['value'].astype('float32')
            new_data['quality'] = new_data['quality'].astype('category')
            data = pd.concat([data, new_data])
            data.drop_duplicates(subset=['dateTime', 'station'], inplace=True)
        return data
        
        
    def get_filename(self, measure: Measure, river):
        return f"{river.lower().replace(' ', '_')}_{measure.value}_raw.parquet"
        
    def update_dataframe(self, df: pd.DataFrame, measure: Measure, river: str):
        for station_name, station_id in self.get_stations_on_river(river)[['label', 'notation']].values:
            print(f"Updating data for station: {station_name}")
            last = df[df['station'] == station_name]['dateTime'].max() if len(df) > 0 else None
            new_measurements = self.get_measure(measure, station_id, last)[['dateTime', 'value', 'quality']]
            new_measurements['station'] = station_name
            new_measurements['station'] = new_measurements['station'].astype('category')
            new_measurements['dateTime'] = pd.to_datetime(new_measurements['dateTime'])
            new_measurements['value'] = new_measurements['value'].astype('float32')
            print(f"Got {len(new_measurements)} new measurements")
            df = pd.concat([df, new_measurements])
        df.drop_duplicates(subset=['dateTime', 'station'], inplace=True)
        return df
            
    def load(self, measure: Measure, river):
        if not os.path.exists(self.DATA_DIR):
            os.mkdir(self.DATA_DIR)
        
        filename = self.get_filename(measure, river)
        filepath = os.path.join(self.DATA_DIR, filename)
        if os.path.exists(filepath):
            print(f"Loading {filepath}")
            df = pd.read_parquet(filepath)
            df['dateTime'] = pd.to_datetime(df['dateTime'])
            df['station'] = df['station'].astype('category')
            df['value'] = df['value'].astype('float32')
        else:
            print(f"Downloading {measure.value} data on: {river}")
            df = self.batch_get_measure_on_river(measure, river)
            df.to_parquet(filepath)
        try:
            df = self.update_dataframe(df, measure, river)
        except Exception as e:
            print(f"Failed to update data: {e}")
        df.to_parquet(filepath)
        return df

def process_hydrology_data(df):
    return df[df['quality'].isin(['Good', 'Unchecked', 'Estimated'])] \
        .pivot(index='dateTime', columns='station', values='value') \
        .resample('15min').interpolate('time', limit_direction='both', limit=2)
        

api = HydrologyApi(max_threads = 2)


## Download Level Data

In [9]:
level_df = api.load(Measure.LEVEL, "River Wear")
level_df.tail()

Loading data\river_wear_level_raw.parquet
{'meta': {'@id': 'http://environment.data.gov.uk/hydrology/id/stations?riverName=River+Wear', 'publisher': 'Environment Agency', 'license': 'http://www.nationalarchives.gov.uk/doc/open-government-licence/version/3/', 'licenseName': 'OGL 3', 'comment': 'Hydrology API for sub-daily data', 'version': '2.0.0', 'hasFormat': ['http://environment.data.gov.uk/hydrology/id/stations.geojson?riverName=River+Wear', 'http://environment.data.gov.uk/hydrology/id/stations.json?riverName=River+Wear', 'http://environment.data.gov.uk/hydrology/id/stations.ttl?riverName=River+Wear', 'http://environment.data.gov.uk/hydrology/id/stations.html?riverName=River+Wear', 'http://environment.data.gov.uk/hydrology/id/stations.csv?riverName=River+Wear', 'http://environment.data.gov.uk/hydrology/id/stations.rdf?riverName=River+Wear'], 'limit': 100}, 'items': [{'@id': 'http://environment.data.gov.uk/hydrology/id/stations/e7d8bbb6-5bba-4057-9f49-a299482c3348', 'label': 'Chester

d:\miniconda3\envs\river-level\Lib\site-packages\pyarrow\pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


,dateTime,value,quality,station
201,2023-11-04 08:15:00,0.878,Unchecked,Durham New Elvet Bridge
202,2023-11-04 08:30:00,0.873,Unchecked,Durham New Elvet Bridge
203,2023-11-04 08:45:00,0.869,Unchecked,Durham New Elvet Bridge
204,2023-11-04 09:00:00,0.870,Unchecked,Durham New Elvet Bridge
205,2023-11-04 09:15:00,0.866,Unchecked,Durham New Elvet Bridge


In [10]:
level_df = process_hydrology_data(level_df)
level_df.tail()

station,Chester Le Street,Durham New Elvet Bridge,Stanhope,Sunderland Bridge,Witton Park
dateTime,,,,,
2023-11-04 08:15:00,1.006,0.878,0.724,0.836,0.910
2023-11-04 08:30:00,1.005,0.873,0.715,0.833,0.901
2023-11-04 08:45:00,1.001,0.869,0.715,0.831,0.902
2023-11-04 09:00:00,0.998,0.870,0.715,0.830,0.898
2023-11-04 09:15:00,0.996,0.866,0.710,0.825,0.894


In [11]:
level_df.to_parquet(os.path.join(HydrologyApi.DATA_DIR, 'river_wear_level.parquet'))
del level_df # Free
print('Saved river_wear_level.parquet')

d:\miniconda3\envs\river-level\Lib\site-packages\pyarrow\pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


Saved river_wear_level.parquet


## Download Flow Data

In [12]:
flow_df = api.load(Measure.FLOW, "River Wear")
flow_df.tail()

{'meta': {'@id': 'http://environment.data.gov.uk/hydrology/id/stations?riverName=River+Wear', 'publisher': 'Environment Agency', 'license': 'http://www.nationalarchives.gov.uk/doc/open-government-licence/version/3/', 'licenseName': 'OGL 3', 'comment': 'Hydrology API for sub-daily data', 'version': '2.0.0', 'hasFormat': ['http://environment.data.gov.uk/hydrology/id/stations.geojson?riverName=River+Wear', 'http://environment.data.gov.uk/hydrology/id/stations.json?riverName=River+Wear', 'http://environment.data.gov.uk/hydrology/id/stations.ttl?riverName=River+Wear', 'http://environment.data.gov.uk/hydrology/id/stations.html?riverName=River+Wear', 'http://environment.data.gov.uk/hydrology/id/stations.csv?riverName=River+Wear', 'http://environment.data.gov.uk/hydrology/id/stations.rdf?riverName=River+Wear'], 'limit': 100}, 'items': [{'@id': 'http://environment.data.gov.uk/hydrology/id/stations/e7d8bbb6-5bba-4057-9f49-a299482c3348', 'label': 'Chester Le Street', 'notation': 'e7d8bbb6-5bba-40

d:\miniconda3\envs\river-level\Lib\site-packages\pyarrow\pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


{'meta': {'@id': 'http://environment.data.gov.uk/hydrology/id/stations?riverName=River+Wear', 'publisher': 'Environment Agency', 'license': 'http://www.nationalarchives.gov.uk/doc/open-government-licence/version/3/', 'licenseName': 'OGL 3', 'comment': 'Hydrology API for sub-daily data', 'version': '2.0.0', 'hasFormat': ['http://environment.data.gov.uk/hydrology/id/stations.geojson?riverName=River+Wear', 'http://environment.data.gov.uk/hydrology/id/stations.json?riverName=River+Wear', 'http://environment.data.gov.uk/hydrology/id/stations.ttl?riverName=River+Wear', 'http://environment.data.gov.uk/hydrology/id/stations.html?riverName=River+Wear', 'http://environment.data.gov.uk/hydrology/id/stations.csv?riverName=River+Wear', 'http://environment.data.gov.uk/hydrology/id/stations.rdf?riverName=River+Wear'], 'limit': 100}, 'items': [{'@id': 'http://environment.data.gov.uk/hydrology/id/stations/e7d8bbb6-5bba-4057-9f49-a299482c3348', 'label': 'Chester Le Street', 'notation': 'e7d8bbb6-5bba-40

d:\miniconda3\envs\river-level\Lib\site-packages\pyarrow\pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


,dateTime,value,quality,station
1548704,2023-10-14 05:00:00,1.81,Unchecked,Stanhope
1548705,2023-10-14 05:15:00,1.81,Unchecked,Stanhope
1548706,2023-10-14 05:30:00,1.81,Unchecked,Stanhope
1548707,2023-10-14 05:45:00,1.81,Unchecked,Stanhope
1548708,2023-10-14 06:00:00,1.81,Unchecked,Stanhope


In [14]:
flow_df = process_hydrology_data(flow_df)
flow_df.tail()

station,Chester Le Street,Stanhope,Sunderland Bridge,Witton Park
dateTime,,,,
2023-10-14 05:00:00,7.38,1.81,5.24,3.78
2023-10-14 05:15:00,7.34,1.81,5.21,3.76
2023-10-14 05:30:00,7.43,1.81,5.21,3.76
2023-10-14 05:45:00,7.34,1.81,5.21,3.76
2023-10-14 06:00:00,7.34,1.81,5.21,3.74


In [ ]:
flow_df.to_parquet(os.path.join(HydrologyApi.DATA_DIR, 'river_wear_flow.parquet'))
del flow_df

## Download Rainfall Data

In [33]:
rainfall_stations = api.get_stations_close_to_with_measure(54.66305556, -1.67611111, 40, Measure.RAINFALL, limit=1000)

print(f"Got {len(rainfall_stations)} rainfall stations")
rainfall_stations.head()

Got 72 rainfall stations


,@id,label,notation,easting,northing,lat,long,type,stationGuid,wiskiID,dateOpened,observedProperty,status,measures,colocatedStation
0,http://environment.data.gov.uk/hydrology/id/st...,Evenwood Gate,1dabd12c-1d2e-4765-ae38-a4d5a121928d,416820,523978,54.610671,-1.741089,[{'@id': 'http://environment.data.gov.uk/flood...,1dabd12c-1d2e-4765-ae38-a4d5a121928d,023164,1997-04-16,[{'@id': 'http://environment.data.gov.uk/refer...,{'@id': 'http://environment.data.gov.uk/flood-...,[{'@id': 'http://environment.data.gov.uk/hydro...,NaN
1,http://environment.data.gov.uk/hydrology/id/st...,ESH WINNING,15202aee-c5fd-404d-9de9-7357174ad10c,420084,542275,54.774977,-1.689302,[{'@id': 'http://environment.data.gov.uk/flood...,15202aee-c5fd-404d-9de9-7357174ad10c,024121,NaN,[{'@id': 'http://environment.data.gov.uk/refer...,{'@id': 'http://environment.data.gov.uk/flood-...,[{'@id': 'http://environment.data.gov.uk/hydro...,NaN
2,http://environment.data.gov.uk/hydrology/id/st...,Harpington Hill Farm,bf61ce31-b20e-4593-85dc-a083133b12ce,433631,526654,54.633882,-1.480513,[{'@id': 'http://environment.data.gov.uk/flood...,bf61ce31-b20e-4593-85dc-a083133b12ce,032822,1987-05-12,[{'@id': 'http://environment.data.gov.uk/refer...,{'@id': 'http://environment.data.gov.uk/flood-...,[{'@id': 'http://environment.data.gov.uk/hydro...,NaN
3,http://environment.data.gov.uk/hydrology/id/st...,Copley,bc34e640-d9ae-4362-8804-25d66ca66e4d,408498,525452,54.624124,-1.869893,[{'@id': 'http://environment.data.gov.uk/flood...,bc34e640-d9ae-4362-8804-25d66ca66e4d,022994,1997-04-02,[{'@id': 'http://environment.data.gov.uk/refer...,{'@id': 'http://environment.data.gov.uk/flood-...,[{'@id': 'http://environment.data.gov.uk/hydro...,NaN
4,http://environment.data.gov.uk/hydrology/id/st...,Tunstall,051f1b2a-6aca-4402-8956-5474ad39b12a,406370,540768,54.761793,-1.902523,[{'@id': 'http://environment.data.gov.uk/flood...,051f1b2a-6aca-4402-8956-5474ad39b12a,022163,1982-01-01,[{'@id': 'http://environment.data.gov.uk/refer...,{'@id': 'http://environment.data.gov.uk/flood-...,[{'@id': 'http://environment.data.gov.uk/hydro...,NaN


In [34]:
rainfall_df = api.batch_get_measure_from_stations(Measure.RAINFALL, rainfall_stations)
rainfall_df

Making request to: https://environment.data.gov.uk/hydrology/data/batch-readings/batch/?measure=1dabd12c-1d2e-4765-ae38-a4d5a121928d-rainfall-t-900-mm-qualifiedMaking request to: https://environment.data.gov.uk/hydrology/data/batch-readings/batch/?measure=15202aee-c5fd-404d-9de9-7357174ad10c-rainfall-t-900-mm-qualified

Got empty CSV
Making request to: https://environment.data.gov.uk/hydrology/data/batch-readings/batch/?measure=bf61ce31-b20e-4593-85dc-a083133b12ce-rainfall-t-900-mm-qualified
Making request to: https://environment.data.gov.uk/hydrology/data/batch-readings/batch/?measure=bc34e640-d9ae-4362-8804-25d66ca66e4d-rainfall-t-900-mm-qualified
Making request to: https://environment.data.gov.uk/hydrology/data/batch-readings/batch/?measure=50568500-9583-4797-8e0f-989df20bfa7b-rainfall-t-900-mm-qualified
Got empty CSV
No new data for station: ESH WINNING


In [ ]:
rainfall_df = process_hydrology_data(rainfall_df)
rainfall_df.tail()

station,Copley,Evenwood Gate,Harpington Hill Farm,Tunstall
dateTime,,,,
2023-11-04 11:15:00,NaN,0.0,0.0,0.0
2023-11-04 11:30:00,NaN,0.0,0.0,0.0
2023-11-04 11:45:00,NaN,0.0,0.0,0.0
2023-11-04 12:00:00,NaN,0.0,0.0,0.0
2023-11-04 12:15:00,NaN,0.0,0.0,0.0


## Combine datasets

In [ ]:
def rename_columns(df, prefix):
    df.columns = [prefix + c for c in df.columns]
    return df

all_data = pd.concat([
    rename_columns(level_df, 'level_'),
    rename_columns(flow_df, 'flow_'),
    rename_columns(rainfall_df, 'rainfall_')
], axis=1)

all.data.head()